In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import time

In [ ]:
# for parallelization if necessary
#from dask import compute, delayed
#from dask.distributed import Client, LocalCluster 
#cl = LocalCluster(n_workers=4)
#client = Client(cl)
#cl

# note: if this throws an error, close jupyter, run "ulimit -n 4096" in terminal, then restart jupyter

In [3]:
# define data location
#location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
location = '/Users/loki/Documents/Data/Forecasting Project/'

In [21]:
# read data from pickle

# what data granularity?
granularity = "daily"

# if we're clustering first, we need to load in sufficient_ data to ensure data equivalency across IDs
if granularity == "daily":
    readdata = pd.read_pickle(location+'peco_sufficient_daily.pkl.zip')
    weather = pd.read_pickle(location+'daily_weather.pkl.zip')
    ids = pd.read_pickle(location+'peco_sufficient_daily_ids.pkl.zip')
elif granularity == "hourly":
    readdata = pd.read_pickle(location+'peco_sufficient_hourly.pkl.zip')
    weather = pd.read_pickle(location+'hourly_weather.pkl.zip')
    ids = pd.read_pickle(location+'peco_sufficient_hourly_ids.pkl.zip')
else:
    print("Granularity selected was not 'daily' or 'hourly'")


In [23]:
data = readdata.join(pd.get_dummies(readdata['REVENUCODE']))
data = data.drop(columns=['DACCOUNTID','DMETERNO','DCUSTOMERID','FUELTYPE','TARIFF','REVENUCODE','TYPE'])
data = pd.merge(data,
                #looking at f statistics, this is the best model for daily data 
                weather[['Dt','HighDB1','HighDB1delta','Cloud1','Cloud1delta']], 
                how="inner", on='Dt')
data = data[data.notnull()]
data = data.dropna()
data.head()

,Dt,ID,Use,Weekday,Holiday,1.0,3.0,5.0,12.0,HighDB1,HighDB1delta,Cloud1,Cloud1delta
0,2017-10-12,"(189118224335924, 732846282212)",1.0,1,0,1,0,0,0,78.0,-11.0,89.166667,3.75
1,2017-10-12,"(8884140853296, 3881418627996)",0.0,1,0,0,1,0,0,78.0,-11.0,89.166667,3.75
2,2017-10-12,"(224154867687260, 730682241528)",1.0,1,0,1,0,0,0,78.0,-11.0,89.166667,3.75
3,2017-10-12,"(357349866155308, 792549695076)",0.0,1,0,0,1,0,0,78.0,-11.0,89.166667,3.75
4,2017-10-12,"(236128365767248, 610644449116)",1.0,1,0,1,0,0,0,78.0,-11.0,89.166667,3.75


## Regression ##

In [ ]:
# run regression per ID tuple
import statsmodels.api as sm
import statsmodels.formula.api as smf
from timeit import default_timer as timer

start = timer()

# run regressions; add equation to table
for idx in list(ids):
    dat = data[data['ID']==idx]
    
    results = smf.ols('Use ~ HighDB1 + HighDB1delta + Cloud1 + Cloud1delta + Weekday + Holiday',
                      data=dat).fit()
    
    print(results.summary())
    
end = timer()
print(end - start) # Time in seconds

In [24]:
# run regression per ID tuple
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from itertools import chain
from timeit import default_timer as timer

start = timer()

#regr = linear_model.Lasso(alpha = 0.1)
regr = linear_model.LinearRegression(fit_intercept=True)

# create table of each cluster's regression equations
rTable = pd.DataFrame(index=(['intercept']+list(data)[3:] + ['MSE','R2']))

for idx in list(ids):
    dat = data[data['ID']==idx]
    
    X = dat.drop(columns=['ID','Dt','Use'])
    y = dat['Use']
    
    # fit the regression model
    regr.fit(X, y)
    
    # extract model coefficients
    intercept = regr.intercept_
    coefs = regr.coef_
    err = mean_squared_error(y, regr.predict(X))
    r2 = regr.score(X,y)
    #print([intercept]+list(chain(coefs))+[err]+[r2])
    rTable[idx] = [intercept]+list(chain(coefs))+[err]+[r2]

rTable = rTable.T 

end = timer()
print(end - start) # Time in seconds

804.9713499


In [25]:
rTable

,intercept,Weekday,Holiday,1.0,3.0,5.0,12.0,HighDB1,HighDB1delta,Cloud1,Cloud1delta,MSE,R2
"(665128346596, 4464578359312)",1.357392,-0.114202,0.096023,-6.701721e-17,2.455504e-16,-3.171744e-16,0.0,-0.010116,-0.010502,-0.003042,-0.003784,0.287948,0.148396
"(819252079392, 609154378812)",1.077917,-0.130911,0.160072,-1.112552e-16,4.074869e-16,-5.270762e-16,0.0,-0.009364,-0.003560,0.000265,0.001733,0.291656,0.114853
"(819288717920, 4465122076620)",2.978055,-0.106184,0.356511,-2.482691e-16,9.094725e-16,-1.175638e-15,0.0,-0.028122,-0.022940,-0.003249,-0.002765,0.306590,0.494300
"(819880484484, 4465122193056)",0.823565,-0.152351,-0.051612,3.571243e-17,-1.309572e-16,1.686374e-16,0.0,0.004072,-0.002949,-0.000312,-0.001684,1.270107,0.011260
"(1284453836940, 4464330738752)",1.077548,-0.044151,-0.616245,4.286467e-16,-1.567111e-15,2.040876e-15,0.0,-0.004289,0.006925,-0.000286,0.000711,0.347642,0.057037
"(1286739708492, 5162227203084)",14.595835,0.073674,0.701485,-4.899280e-16,1.800719e-15,-2.298759e-15,0.0,-0.160278,-0.122750,-0.019434,-0.013691,1.724347,0.845014
"(1475598667620, 4968015331728)",158.666200,1.335422,3.963711,-2.782040e-15,1.027388e-14,-1.288143e-14,0.0,-1.782297,-1.385993,-0.253135,-0.159745,151.144254,0.885335
"(1631154446780, 4465145519068)",1.412648,0.164977,-0.016351,1.142820e-17,-4.079470e-17,5.790576e-17,0.0,-0.015071,-0.003460,-0.002429,0.000857,0.238139,0.282295
"(1633677808960, 4465140978064)",0.489464,-0.038519,-0.100994,7.018969e-17,-2.563709e-16,3.350346e-16,0.0,-0.004478,-0.002487,0.000567,0.001746,0.185203,0.047772
"(2680127457516, 4465121533252)",3.442847,0.023415,0.467954,-3.259257e-16,1.193319e-15,-1.545600e-15,0.0,-0.028270,-0.027397,-0.006934,-0.005449,0.590917,0.359975


In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

rTable_scaled = pd.DataFrame(scaler.fit_transform(rTable), columns=rTable.columns)
#rTable_scaled.to_csv(location+'daily_home_clusters_scaled.csv')
rTable_scaled

,intercept,Weekday,Holiday,1.0,3.0,5.0,12.0,HighDB1,HighDB1delta,Cloud1,Cloud1delta,MSE,R2
0,-0.120765,-0.035717,-0.002425,0.003032,-0.002851,0.001748,-0.000461,0.123334,0.103239,0.119871,0.095843,-0.023646,-2.945613
1,-0.122367,-0.036535,0.001851,-0.001220,0.001406,-0.002494,-0.000461,0.123794,0.107620,0.134668,0.137038,-0.023646,-3.108933
2,-0.111474,-0.035325,0.014966,-0.014389,0.014604,-0.015601,-0.000461,0.112325,0.095388,0.118943,0.103452,-0.023645,-1.261395
3,-0.123825,-0.037584,-0.012282,0.012906,-0.012750,0.011566,-0.000461,0.132009,0.108006,0.132085,0.111522,-0.023639,-3.613331
4,-0.122369,-0.032290,-0.049981,0.050673,-0.050509,0.049402,-0.000461,0.126897,0.114238,0.132201,0.129402,-0.023645,-3.390443
5,-0.044868,-0.026526,0.037999,-0.037616,0.038037,-0.038299,-0.000461,0.031520,0.032390,0.046519,0.021865,-0.023635,0.446239
6,0.781096,0.035202,0.255805,-0.257924,0.260810,-0.252167,-0.000461,-0.960234,-0.764950,-0.999251,-1.068747,-0.022583,0.642562
7,-0.120448,-0.022059,-0.009928,0.010572,-0.010380,0.009328,-0.000461,0.120304,0.107684,0.122613,0.130499,-0.023646,-2.293653
8,-0.125741,-0.032015,-0.015579,0.016220,-0.016048,0.014928,-0.000461,0.126781,0.108298,0.136019,0.137135,-0.023646,-3.435555
9,-0.108809,-0.028985,0.022407,-0.021853,0.022067,-0.023078,-0.000461,0.112234,0.092575,0.102454,0.083409,-0.023643,-1.915427


### Cluster ###

In [ ]:
# PCA
pca2 = PCA(n_components=2, whiten=True)
X_pca_1cw = pca1.fit_transform(data_scaled)
plt.scatter(X_pca_1cw[:,0], X_pca_1cw[:,1], c=iris.target,
alpha=0.8, s=60, marker='o', edgecolors='white')
plt.show()
pca_2cw.explained_variance_ratio_.sum() 

In [ ]:
# visualize cluster identities using PCA on weight vector

In [ ]:
# visualize rate-code identities using PCA on weight vector

In [ ]:
# append cluster ID to daily/hourly set

In [ ]:
# close dask cluster (if used)
#cl.close